In [5]:
import requests
import gzip as nik
import random as quentin
import solution

ModuleNotFoundError: No module named 'solution'

In [2]:
url = "https://roboats.virtualregatta.com/api/"

def post_login() :
    
    r = requests.post(url+"login",
                  json={"email" : "malo.adler@telecom-paris.fr",
                            "password" : "wonderduck",
                            "raceId" : 531,
                            "legNum" : 1} )
    
    if(r.status_code != 200) :
        raise SystemExit("Login failed : " + str(r.status_code))
        
    return(r.headers['token'])

############################################################################################################################

def get_info_slow(token) :
    
    r = requests.get(url+"infos/slow",
                  headers={"token" : token })
    
    if(r.status_code != 200) :
        raise SystemExit("Get info slow failed : " + str(r.status_code))
        
    return(r.text)

############################################################################################################################
def get_info_fast(token) :
    
    r = requests.get(url+"infos/fast",
                  headers={"token" : token })
    
    if(r.status_code != 200) :
        raise SystemExit("Get info fast failed : " + str(r.status_code))
        
    return(r.text)

############################################################################################################################

def put_boat_action(token , json) :
    
    r = requests.put(url+"boat/actions",
                    headers={"token" : token },
                    json=json)
    
    if(r.status_code != 200) :
        raise SystemExit("Put boat action failed : " + str(r.status_code))
        
    return(r)

############################################################################################################################

def change_heading(token , deg , lock_sail) :
    
    if deg < 0 or deg > 360 :
        raise SystemExit("Degree should be between 0 and 360")
        
    json = {
        "tsLegStart": 1640010600000,
        "actions" : [
            {
                "type": "heading",
                "values" : {
                "deg": deg,
                "autoTwa": lock_sail
                }
            }
        ]
    }
    
    r = put_boat_action(token , json)

############################################################################################################################

def change_sail(token , sailId) :
    
    json = {
        "tsLegStart": 1640010600000,
        "actions" : [
            {
                "type": "sail",
                "values" : {
                    "sailId" : sailId
                }
            }
        ]
    }
    
    r = put_boat_action(token , json)
    
############################################################################################################################

def place_waypoint(token , wpid , lon , lat) :
    
    json = {
        "tsLegStart": 1640010600000,
        "actions" : [
            {
                "type": "wp",
                "values" : {
                    "nextWpIdx": wpid + 1,
                    "coordinates": [
                        {
                            "idx": wpid,
                            "pos" : {
                                "lon": lon,
                                "lat": lat
                            }
                        }
                    ]
                }
            }
        ]
    }
    
    r = put_boat_action(token , json)

In [3]:
token = post_login()
change_heading(token , 56 , False)
change_sail(token , 11)
info_slow = get_info_slow(token)
info_fast = get_info_fast(token)
#place_waypoint(token , 1 , 48 , -7)

In [4]:
r = requests.get(url = 'https://static.virtualregatta.com/ressources/maps/dalles/vro2k16/1/0/4/1_-4_44.deg%27')
file = r.content

print(file)
print(r.encoding)
nik.open(file,mode = 'rb' ,encoding='latin-1')

b'<?xml version="1.0" encoding="UTF-8"?>\n<Error><Code>AccessDenied</Code><Message>Access Denied</Message><RequestId>WATC4NAQPWZMW5B1</RequestId><HostId>oiW48Am52XyddDYM058on46GzbY55ApbyWVZVrzRCvGc1D6osFOivAVvOjJf062pH5V0e6KSocg=</HostId></Error>'
None


ValueError: Argument 'encoding' not supported in binary mode